<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/Clase_Aprendizaje_Profundo/blob/main/L08_Transferencia_de_Aprendizaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje
## Dr. Carlos Villaseñor

# Parte 1. Introducción a la transferencia de aprendizaje

Importar bibliotecas



In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras

Importar MobileNet base pero descartaremos las últimas capa
(Revisa este [enlace](https://keras.io/api/applications/) )

In [ ]:
base_model = keras.applications.MobileNetV2(weights='imagenet',include_top=False)

Crear nueva Red Neuronal

In [ ]:
x=base_model.output
x=keras.layers.GlobalAveragePooling2D()(x)
x=keras.layers.Dense(256,activation='relu')(x)
x=keras.layers.Dense(64,activation='relu')(x)
preds=keras.layers.Dense(3, activation='softmax')(x)

Pegar Redes Neuronales

In [ ]:
model=keras.Model(inputs=base_model.input, outputs=preds)

Definir que capas se entrenan

In [ ]:
freeze_point = 5
for layer in model.layers[:-freeze_point]:
    layer.trainable=False
for layer in model.layers[-freeze_point:]:
    layer.trainable=True

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [ ]:
!wget "https://github.com/Dr-Carlos-Villasenor/Taller_CNN/raw/main/train.zip"

In [ ]:
!unzip train.zip

Crear generador desde carpeta

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Compilar Modelo y entrenar desde carpeta

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          epochs=6)

# Parte 2 Tarea
El código anterior se demuestra como podemos hacer Deep Learning incluso teniendo pocos datos, gracias a que el modelo está preentrenado.

Sin embargo hay algunos pasos mal en el código anterior.

Paso 1 Dataset: Para completar esta actividad primeramente mejoraremos el dataset anterior. Para eso elabora lo siguiente:

* Descarga la carpeta de entrenamiento,
* Elige dos clases nuevas de tu preferencia (e.g.: humanos, lobos, mariposas etc..),
* Genera otra carpeta llamada 'test',
* Dentro de las carpetas 'train' y 'test' crea las carpetas para las 5 clases.
* Asegurate de que por lo menos en cada subcarpeta de 'train' tengas 50 muestras.
* Asegurate de que por lo menos cada subcarpeta de 'test' tenga al menos 15 muestras.
* Carga las carpetas a tu colab para trabajar con ellas.

Paso 2 Entrenamiento: Ahora bien modificaremos el código de este libro de colab para trabajar con el data set nuevo. Para que sea metodologicamente correcto haz lo siguiente:

* Modifica el bloque de image data generator, ya que necesitamos uno más para la carpeta 'test', con lo que tendras un 'test_datagen' y un 'test_generator'.
Por ejemplo:

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(224, 224),
        batch_size = 15*5,
        class_mode='categorical')

* Modifica la salida de la red para tenga 5 neuronas en su última capa (una por cada clase). Por ejemplo:


In [ ]:
preds=keras.layers.Dense(5, activation='softmax')(x)

* Reentrena tu red, usa los datos del test como conjunto de validación, por ejemplo:

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=120,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

* Modifica tu red paera obtener un buen desempeño (que no esté sobreentrenada ni subentrenada).
* Obtene la matriz de confusión del conjunto de 'test'.